In [1]:
import os

import pymongo

from splinter import Browser
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait # available since 2.4.0
from selenium.webdriver.support import expected_conditions as EC # available since 2.26.0

import pandas as pd

import re
from pprint import pprint
from datetime import datetime
import time

import sqlalchemy
from sqlalchemy import create_engine, MetaData, inspect, func
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, ForeignKey, Date, DateTime, Boolean, BigInteger
from sqlalchemy.orm import sessionmaker, relationship


import sys
sys.path.append('C:\\Users\\soria\\Documents\\WashUDataDocuments\\HwActivites\\YouTube_Project\\')
from config import YOUTUBE_DATA_API_KEY
import pandas as pd
import json
import os
from pprint import pprint

from apiclient.discovery import build
from apiclient.errors import HttpError
from oauth2client.tools import argparser

Connect to mongoDb client and create database name

In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
mongoDb = client.youtube
#   collection.insert_one(test_programs)
#   collection = db[program_name]
#   collection.update_one({}, {'$set': kdhx_dict}, upsert=True)


Connect to local mySql, create engine, declare base

In [3]:
rds_connection_string = "root:Password@127.0.0.1"
engine = create_engine(f'mysql://{rds_connection_string}')
engine.execute('CREATE DATABASE IF NOT EXISTS youtube_db')
engine.execute('USE youtube_db')
Base = declarative_base()


base class for region_codes

In [4]:
class Region_Codes(Base):
    __tablename__ = 'region_codes'
    id = Column(Integer, primary_key=True)
    region = Column(String(256))
    region_code = Column(String(256))
    
    def __init__(self, region, region_code):
        self.region = region
        self.region_code = region_code
        
        
    def __repr__(self):
        return f"id={self.id}, region={self.region}, region_code={self.region_code}"

base class for category_ids

In [5]:
class Category_Ids(Base):
    __tablename__ = 'category_ids'
    id = Column(Integer, primary_key=True)
    region_codes_table_id = Column(Integer, ForeignKey('region_codes.id'))
    category_title = Column(String(256))
    category_id = Column(String(256))
    category_is_assignable = Column(Boolean)
    region_code = Column(String(256))
    
    def __init__(self, category_title, category_id, category_is_assignable, region_code):
        self.category_title = category_title
        self.category_id = category_id
        self.category_is_assignable = category_is_assignable
        self.region_code = region_code
        
    def __repr__(self):
        return f"id={self.id}, category_title={self.category_title}, category_id={self.category_id}"
  

base class for topic_ids

In [6]:
class Topic_Ids(Base):
    __tablename__ = 'topic_ids'
    id = Column(Integer, primary_key=True)
    topic_name = Column(String(256))
    topic_id = Column(String(256))
    topic_heading = Column(String(256))
    
    def __init__(self, topic_name, topic_id, topic_heading):
        self.topic_name = topic_name
        self.topic_id = topic_id
        self.topic_heading = topic_heading
        
    def __repr__(self):
        return f"id={self.id}, topic_name={self.topic_name}, topic_id={self.topic_id}, topic_heading={self.topic_heading}"

base class for channels

In [7]:
class Channel(Base):
    __tablename__ = 'channels'
    id = Column(Integer, primary_key=True)
    topic_ids_table_id = Column(Integer, ForeignKey('topic_ids.id'))
    channel_title = Column(String(256))
    channel_url = Column(String(256))
    channel_id = Column(String(256))
    channel_keywords = Column(String(256))
    channel_description = Column(String(1024))
    subscribers = Column(BigInteger)
    video_count = Column(Integer)
    view_count = Column(BigInteger)
    country = Column(String(256))
    join_date = Column(Date)
    topic_ids = Column(String(256))
    cc_category = Column(String(256))
    thumbnail_url = Column(String(256))
    
    def __init__(self, topic_ids_table_id, channel_title, channel_url, channel_id, channel_keywords, channel_description,\
                 subscribers, video_count, view_count, country, join_date, topic_ids, cc_category, thumbnail_url):
        self.topic_ids_table_id = topic_ids_table_id
        self.channel_title = channel_title
        self.channel_url = channel_url
        self.channel_id = channel_id
        self.subscribers = subscribers
        self.video_count = video_count
        self.view_count = view_count
        self.country = country
        self.join_date = join_date
        self.join_dto = join_dto
        self.topic_ids = topic_ids
        self.cc_category = cc_category
        self.thumbnail_url = thumbnail_url
        
        
    def __repr__(self):
        return f"id={self.id}, channel_title={self.channel_title}, url={self.channel_url}"

base class for videos

In [8]:
class Videos(Base):
    __tablename__ = 'videos'
    id = Column(Integer, primary_key=True)
    channels_table_id = Column(Integer, ForeignKey('channels.id'))
    topic_ids_table_id = Column(Integer, ForeignKey('topic_ids.id'))
    category_ids_table_id = Column(Integer, ForeignKey('category_ids.id'))
    video_title = Column(String(256))
    video_description = Column(String(1024))
    view_count = Column(BigInteger)
    duration = Column(Integer) # https://stackoverflow.com/questions/15596753/how-do-i-get-video-durations-with-youtube-api-version-3
    like_count = Column(Integer)
    dislike_count = Column(Integer)
    favorite_count = Column(Integer)
    comment_count = Column(Integer)
    published_at = Column(DateTime)
    published_at_dto = Column(DateTime)
    tags = Column(String(256))
    dimension = Column(String(256))
    definition = Column(String(256))
    caption = Column(Boolean)
    licensedContent = Column(Boolean)
    license = Column(String(256))
    
    def __init__(self, channels_table_id, topic_ids_table_id, category_ids_table_id, video_title, video_description,\
                 view_count, duration, like_count, dislike_count, favorite_count, comment_count, published_at,\
                 published_at_dto, tags, dimension, definition, caption, licensedContent, license):
        self.channels_table_id = channels_table_id
        self.topic_ids_table_id = topic_ids_table_id
        self.category_ids_table_id = category_ids_table_id
        self.video_title = video_title
        self.view_count = view_count
        self.duration = duration
        self.like_count = like_count
        self.dislike_count = dislike_count
        self.favorite_count = favorite_count
        self.comment_count = comment_count
        self.published_at = published_at
        self.published_at_dto = published_at_dto
        self.tags = tags
        self.dimension = topic_ids
        self.definition = definition
        self.caption = caption
        self.licensedContent = licensedContent
        self.license = license
        
        
    def __repr__(self):
        return f"id={self.id}, video_title={self.video_title}, view_count={self.view_count}"

In [9]:
Session = sessionmaker(bind=engine)
sqlDbSession = Session()

Create a "Metadata" Layer That Abstracts our SQL Database

In [10]:


Base.metadata.create_all(engine)

# Use this to clear out the db
# ----------------------------------
# Base.metadata.drop_all(engine)

point splinter to chromedriver, assign browser object to variable

In [11]:
executable_path = {'executable_path': '/Users/soria/Anaconda3/Scripts/chromedriver'}
browser = Browser('chrome', **executable_path)


declare url to gist site hosting topic id list, visit and wait for necessary element to load

In [12]:
url = "https://gist.github.com/stpe/2951130dfc8f1d0d1a2ad736bef3b703"
driver = webdriver.Chrome()
driver.get(url)
wait = WebDriverWait(driver, 10)
element = wait.until(EC.element_to_be_clickable((By.ID,'user-content-other-topics')))


In [13]:
client.list_database_names()

['ClassDB',
 'admin',
 'config',
 'craigslist_app',
 'craigslist_db',
 'fruits_db',
 'kdhx',
 'kdhx_shows',
 'local',
 'mars_app',
 'nhl_db',
 'store_inventory',
 'team_db',
 'travel_db',
 'weather_app',
 'youtube']

In [14]:
mongoDb.list_collection_names()

['youtube_topicIds', 'topicId_gist_html']

In [15]:
topicId_gist_html = mongoDb.topicId_gist_html
topicId_gist_html

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'youtube'), 'topicId_gist_html')

In [21]:
collection = mongoDb.topicId_gist_html
if mongoDb.topicId_gist_html.find_one():
    html = mongoDb.topicId_gist_html.find_one()['channel_crawler_html']
else:
    html = driver.page_source
topic_id_html_dict = {'topicId_gist_url': driver.current_url, 'channel_crawler_html':html}
collection.update_one({}, {'$set': topic_id_html_dict}, upsert=True)
topic_soup = bs(html, 'html.parser')
heading_list = []
topic_headings = topic_soup.find_all('h2')
for heading in topic_headings:
    heading_list.append(heading.text)
topics_dict = {}
topic_dict_list = []
topics = topic_soup.find_all('li')
for index, topic in enumerate(topics[4:20]):
    text = topic.text
    topic_tuple = re.split(r'\s\s\s', text)
    collection = mongoDb.youtube_topicIds
    topic_name = topic_tuple[1]
    topic_id = topic_tuple[0]
    topic_heading = heading_list[0]
    topics_dict.setdefault(topic_heading, [])
    topics_dict[topic_heading].append({'topic_name': topic_name, 'topic_id': topic_id, 'topic_heading': topic_heading})
    topic_dict_list.append(topics_dict)
    collection.update_one({}, {'$set': topics_dict}, upsert=True)
    sqlDbSession.merge(Topic_Ids(topic_name, topic_id, topic_heading))
    sqlDbSession.commit()
for index, topic in enumerate(topics[20:31]):
    text = topic.text
    topic_tuple = re.split(r'\s\s\s', text)
    collection = mongoDb.youtube_topicIds
    topic_name = topic_tuple[1]
    topic_id = topic_tuple[0]
    topic_heading = heading_list[0]
    topics_dict.setdefault(topic_heading, [])
    topics_dict[topic_heading].append({'topic_name': topic_name, 'topic_id': topic_id, 'topic_heading': topic_heading})
    topic_dict_list.append(topics_dict)
    collection.update_one({}, {'$set': topics_dict}, upsert=True)
    sqlDbSession.merge(Topic_Ids(topic_name, topic_id, topic_heading))
    sqlDbSession.commit()
for index, topic in enumerate(topics[31:45]):
    text = topic.text
    topic_tuple = re.split(r'\s\s\s', text)
    collection = mongoDb.youtube_topicIds
    topic_name = topic_tuple[1]
    topic_id = topic_tuple[0]
    topic_heading = heading_list[0]
    topics_dict.setdefault(topic_heading, [])
    topics_dict[topic_heading].append({'topic_name': topic_name, 'topic_id': topic_id, 'topic_heading': topic_heading}) 
    topic_dict_list.append(topics_dict)
    collection.update_one({}, {'$set': topics_dict}, upsert=True)
    sqlDbSession.merge(Topic_Ids(topic_name, topic_id, topic_heading))
    sqlDbSession.commit()
for index, topic in enumerate(topics[45:50]):
    text = topic.text
    topic_tuple = re.split(r'\s\s\s', text)
    collection = mongoDb.youtube_topicIds
    topic_name = topic_tuple[1]
    topic_id = topic_tuple[0]
    topic_heading = heading_list[0]
    topics_dict.setdefault(topic_heading, [])
    topics_dict[topic_heading].append({'topic_name': topic_name, 'topic_id': topic_id, 'topic_heading': topic_heading}) 
    topic_dict_list.append(topics_dict)
    collection.update_one({}, {'$set': topics_dict}, upsert=True)
    sqlDbSession.merge(Topic_Ids(topic_name, topic_id, topic_heading))
    sqlDbSession.commit()
for index, topic in enumerate(topics[50:61]):
    text = topic.text
    topic_tuple = re.split(r'\s\s\s', text)
    collection = mongoDb.youtube_topicIds
    topic_name = topic_tuple[1]
    topic_id = topic_tuple[0]
    topic_heading = heading_list[0]
    topics_dict.setdefault(topic_heading, [])
    topics_dict[topic_heading].append({'topic_name': topic_name, 'topic_id': topic_id, 'topic_heading': topic_heading}) 
    topic_dict_list.append(topics_dict)
    collection.update_one({}, {'$set': topics_dict}, upsert=True)
    sqlDbSession.merge(Topic_Ids(topic_name, topic_id, topic_heading))
    sqlDbSession.commit()
for index, topic in enumerate(topics[61:63]):
    text = topic.text
    topic_tuple = re.split(r'\s\s\s', text)
    collection = mongoDb.youtube_topicIds
    topic_name = topic_tuple[1]
    topic_id = topic_tuple[0]
    topic_heading = heading_list[0]
    topics_dict.setdefault(topic_heading, [])
    topics_dict[topic_heading].append({'topic_name': topic_name, 'topic_id': topic_id, 'topic_heading': topic_heading})
    topic_dict_list.append(topics_dict)
    collection.update_one({}, {'$set': topics_dict}, upsert=True)
    sqlDbSession.merge(Topic_Ids(topic_name, topic_id, topic_heading))
    sqlDbSession.commit()

In [17]:
topic_results = sqlDbSession.query(Topic_Ids.topic_name, Topic_Ids.topic_id, Topic_Ids.topic_heading)\
                    .all()
topic_name = [result[0] for result in topic_results]
topic_ids = [result[1] for result in topic_results]
topic_headings = [result[2] for result in topic_results]

In [23]:
topic_df = pd.DataFrame(topic_results, columns=['topic_name', 'topic_ids', 'topic_headings'])
# topic_df.set_index('topic_name', inplace=True, )
topic_df.head()
topic_df.to_csv('./../../../youtube_project_data/youtube_topic_ids.csv')

In [ ]:
sqlDbSession.close()

In [ ]:
rds_connection_string = "root:Password@127.0.0.1"
engine = create_engine(f'mysql://{rds_connection_string}')
# engine.execute('CREATE DATABASE IF NOT EXISTS youtube_db')
engine.execute('USE youtube_db')
topic_stmt = sqlDbSession.query(Topic_Ids).statement
topic_df2 = pd.read_sql_query(topic_stmt, con=engine)
topic_df2.head()

In [24]:
Session = sessionmaker(bind=engine)
sqlDbSession = Session()

In [25]:
DEVELOPER_KEY = YOUTUBE_DATA_API_KEY
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"


In [26]:
youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
    developerKey=DEVELOPER_KEY)

list of region codes

In [31]:
mongoDb.regionCode_ytApi_resonse.find_one()

{'_id': ObjectId('5c610b40e04415cc2bf7a7a9'),
 'region_code_ytApi_resonse': {'kind': 'youtube#i18nRegionListResponse',
  'etag': '"XpPGQXPnxQJhLgs6enD_n8JR4Qk/OOaJ0ui26Q9yT9f0czHbTKo0fus"',
  'items': [{'kind': 'youtube#i18nRegion',
    'etag': '"XpPGQXPnxQJhLgs6enD_n8JR4Qk/R_GB1d7CQi3LIpoHKbakFDisvoA"',
    'id': 'DZ',
    'snippet': {'gl': 'DZ', 'name': 'Algeria'}},
   {'kind': 'youtube#i18nRegion',
    'etag': '"XpPGQXPnxQJhLgs6enD_n8JR4Qk/w6ci5tJWSaqFmjn3xsM2loOjo2o"',
    'id': 'AR',
    'snippet': {'gl': 'AR', 'name': 'Argentina'}},
   {'kind': 'youtube#i18nRegion',
    'etag': '"XpPGQXPnxQJhLgs6enD_n8JR4Qk/LEM-PlciSxmD60HgrhMWaA7tzaw"',
    'id': 'AU',
    'snippet': {'gl': 'AU', 'name': 'Australia'}},
   {'kind': 'youtube#i18nRegion',
    'etag': '"XpPGQXPnxQJhLgs6enD_n8JR4Qk/FkK6IWxiZ9A5dzRCN0QIa6Bj16s"',
    'id': 'AT',
    'snippet': {'gl': 'AT', 'name': 'Austria'}},
   {'kind': 'youtube#i18nRegion',
    'etag': '"XpPGQXPnxQJhLgs6enD_n8JR4Qk/tWYVnJRRw31zLtf0LtIx0Qjg3gc"',
  

In [ ]:
if mongoDb.topicId_gist_html.find_one():
    html = mongoDb.topicId_gist_html.find_one()['channel_crawler_html']

In [32]:
collection = mongoDb.regionCode_ytApi_resonse
if mongoDb.regionCode_ytApi_resonse.find_one():
    region_code_search_response = mongoDb.regionCode_ytApi_resonse.find_one()['region_code_ytApi_resonse']
else:
    region_code_search_response = youtube.i18nRegions().list(
    part="snippet,id"
        
    ).execute()
region_code_response_dict = {'region_code_ytApi_resonse': region_code_search_response}
collection.update_one({}, {'$set': region_code_response_dict}, upsert=True)
region_code_dict_list = []
for item in region_code_search_response['items']:
    region = item['snippet']['name']
    region_code = item['snippet']['gl']    
    region_code_dict = {'region': region, 'region_code': region_code}
    collection = mongoDb.youtube_regionCodes
    region_code_dict_list.append(region_code_dict)
    collection.update_one({}, {'$set': region_code_dict}, upsert=True)
    sqlDbSession.add(Region_Codes(region, region_code))
    sqlDbSession.commit()
    topics_dict.setdefault(topic_heading, [])
    topics_dict[topic_heading].append({'topic_name': topic_name, 'topic_id': topic_id, 'topic_heading': topic_heading})

In [ ]:
region_code_stmt = sqlDbSession.query(Region_Codes).statement
region_code_df = pd.read_sql_query(region_code_stmt, con=engine)
region_code_df.head()

In [ ]:
region_code_dict_list

In [ ]:
for i, item in enumerate(region_code_dict_list):
    item.setdefault('categories', {})
    item['categoroes'] = i
region_code_dict_list

category_Id search

In [ ]:
for region in region_code_dict_list:
    region_code = region['region_code']
    video_category_search_response = youtube.videoCategories().list(
        regionCode=region_code,
        part="snippet,id"
    ).execute()
    coll_name = f"{region_code}_video_cateogory_ytApi_resonse"
    collection = mongoDb.coll_name
    video_category_response_dict = {coll_name: video_category_search_response}
    collection.update_one({}, {'$set': video_category_response_dict}, upsert=True)
    coll_name = f"{region_code}_youtube_category_ids"
    collection = mongoDb.coll_name
    region_category_list = []
    for item in video_category_search_response['items']:
        category_title = item['snippet']['title']
        category_id = item['id']
        category_is_assignable = item['snippet']['assignable']
        category_id_dict = {'category_title': category_title, 'category_id': category_id,\
                            'category_is_assignable': category_is_assignable, 'region_code': region_code}
        region_category_list.append(category_id_dict)
        collection.update_one({}, {'$set': category_id_dict}, upsert=True)
        sqlDbSession.add(Category_Ids(category_title, category_id, category_is_assignable, region_code)) 
        sqlDbSession.commit()
    region['categories'] = region_category_list
                         


In [ ]:
sqlDbSession.commit()

In [ ]:
region_code_dict_list

In [ ]:
categoryKeys = []
for k, v in video_category_search_response['items'][0]['snippet'].items():
    categoryKeys.append(k)
categoryKeys = categoryKeys
categoryKeys

In [ ]:

for item in video_category_search_response['items']:
    print(f"id: {item['id']}")
    print(f"  title: {item['snippet']['title']}")
    print(f"  assignable: {item['snippet']['assignable']}")
    print("========")
    

In [ ]:
sqlDbSession.close()